<a href="https://colab.research.google.com/github/PedroHenriqueS22/Praticas_Analise_Dados_DNC/blob/main/Case6_AutoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# O AutoML (Automated Machine Learning)

## Técnica que permite a seleção e otimização automática de modelos de Machine Learning

* Observação 1: O modelo selecionado pelo AutoML foi inferior ao melhor modelo selecionado no Case5_Modeling. Isso se dá devido o Auto-Sklearn só avaliar os modelos presentes no scikit-learn,porém no Case 5 utilizamos modelos mais avançados como o XGBoost que está em outra biblioteca e sendo assim não foi testada.

* Observação 2: O AutoML funcionou como esperado, visto que encontrou um modelo do scikit-learn melhor que os modelos Linear Regression e SVR que foram testados no Case 5.













In [ ]:
!pip install Cython==0.29.36
!pip uninstall scipy -y
!pip install scipy==1.9
!pip uninstall pyparsing -y
!pip install pyparsing==2.4
!pip uninstall scikit_learn -y

In [ ]:
!pip uninstall imbalanced-learn -y
!pip uninstall mlxtend -y
!pip uninstall yellowbrick -y

In [ ]:
!pip install scikit-learn==0.24.2 --no-build-isolation

In [ ]:
!pip install auto-sklearn

In [10]:
import autosklearn.regression
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [3]:
data_url = "http://lib.stat.cmu.edu/datasets/boston"
housing = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([housing.values[::2, :], housing.values[1::2, :2]])
target = housing.values[1::2, 2]

columns_names = ["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT", "MEDV"]

df = pd.DataFrame(data, columns=columns_names[:-1])

In [4]:
# Características das casas
X = data

# Preço conhecido das casas
y = target

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [6]:
# Criando variável com as configurações desejadas, para ser testada com vários modelos e nos "dizer" qual o melhor modelo
AutoML = autosklearn.regression.AutoSklearnRegressor(time_left_for_this_task=120, per_run_time_limit = 30,metric= autosklearn.metrics.mean_squared_error)

In [7]:
# Treinando modelos com a massa de teste
AutoML_Train = AutoML.fit(X_train, y_train)

In [8]:
# Verificando quais modelos foram executados
AutoML_Train.leaderboard()

,rank,ensemble_weight,type,cost,duration
model_id,,,,,
6,1,0.42,gaussian_process,10.350982,26.092179
9,2,0.34,extra_trees,10.947156,2.023753
5,3,0.24,ard_regression,12.624760,0.715317


In [9]:
# Testando o modelo número 1 do rank
yAutoML= AutoML_Train.predict(X_test)
print(yAutoML)

[24.18826675 33.49907398 18.97885418 23.15297842 16.59366608 21.1917944
 17.56312037 15.49022627 22.03917742 20.49093723 20.41005468 17.14800882
 12.26988268 21.91415834 18.86107445 23.05305147 19.15137291  9.77066088
 46.19717121 14.6405344  25.02059364 26.76375008 14.22049737 22.88788891
 17.38401747 17.62569189 21.37897253 13.43870664 19.47731686 20.45330334
 20.24053621 24.1242013  19.72703075 26.97072029 14.79222536 16.0358448
 33.58189678 19.57836056 21.7660594  24.14179325 18.34436035 30.6452775
 49.62209511 18.40232468 23.85965395 16.45518303 14.66073704 25.62883186
 19.11239958 26.83336782 19.09010363 36.1045208  15.95915675 26.22596788
 42.92492294 21.95290565 15.74659348 32.06930351 23.76223516 16.81402183
 25.06258249 34.86586857 31.01214504 17.51027513 23.83161831 18.7325983
 15.33734608 24.07662106 29.3754158  15.03249454 19.90723705 25.79474735
 13.38940954 20.89634514 22.75830221  8.70795441 19.97077179 47.85875607
 13.20453715 18.07377005 20.38718939 12.1902566  18.745

In [11]:
MSE = mean_squared_error(y_test, yAutoML) # MSE com y_test, pois foi utilizado o X_test para fazer a predição, e o valor predito

In [14]:
print(f'MSE AutoML: {MSE*1000:f}')
print(f'RMSE AutoML: {np.sqrt(MSE)*1000:f}') # Cálculo do RMSE indicando que os erros dos preços foram em média de

MSE AutoML: 11433.458272
RMSE AutoML: 3381.339716
